In [ ]:
import openai
import os
import random

class AIAgent:
    def __init__(self, name, api_key=None):
        self.name = name
        self.memory = []
        self.api_key = api_key

    def receive_message(self, message):
        self.memory.append(message)

class AIAgentWithChatGPT(AIAgent):
    def __init__(self, name, api_key=None):
        super().__init__(name, api_key)
        openai.api_key = api_key or os.environ.get("OPENAI_API_KEY")

    def get_chatgpt_response(self, prompt):
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                {
                    "role": "user",
                    "content": prompt,
                }
            ],
        )
        
        answer = response.choices[0].message.content
        # print(f"{self.name} got the response: {answer}")
        return answer

def explain_rules(agents):
    for agent in agents:
        agent.receive_message(
            "You will play a game of 20 Questions. One of you will be the answerer, and the other will be the guesser. "
            "The game is turn-based, with the guesser asking up to 20 yes/no questions. One turn consists of one question and one answer. "
            "The guesser can only guess the object after all 20 questions have been asked and answered."
        )

def play_20_questions(answerer, guesser):
    object_to_guess = input("User, please enter the object to be guessed: ")
    answerer.receive_message(object_to_guess)

    print(f"{answerer.name} is the answerer.")
    print(f"{guesser.name} is the guesser.")

    previous_answers = []

    for i in range(20):
        # Asks a question while retaining context through prompts.
        context = f"The object is unknown. Here are the previous answers: {'; '.join(previous_answers)}. "
        prompt = context + "Based on this, suggest the next single yes/no question to narrow down the possibilities. ONLY ASK ONE QUESTION."
        question = guesser.get_chatgpt_response(prompt)

        # Print the question in the desired format
        print(f"Question {i+1}: {question}")

        # Answerer's response should be yes or no based on the object
        response = answerer.get_chatgpt_response(f"The object is {object_to_guess}. Answer yes or no to this question: {question}")
        
        # Print the answer in the desired format
        print(f"Answer: {response.strip()}")

        previous_answers.append(f"Q: {question} - A: {response.strip()}")

        input("Press Enter to continue to the next question...")

    # After all 20 questions, ask ChatGPT to guess the object
    guess_prompt = (
        f"Based on the 20 questions and answers: {'; '.join(previous_answers)}, "
        "what is your guess for the object?"
    )
    guess_response = guesser.get_chatgpt_response(guess_prompt)

    print(f"ChatGPT's guess: {guess_response}")
    if object_to_guess.lower() in guess_response.lower():
        print(f"{guesser.name} guessed the object correctly!")
    else:
        print(f"{guesser.name}'s guess was incorrect. The object was: {object_to_guess}")

# API Key for OpenAI (replace with your own or set it in the environment)
api_key = ""

# Creats two AI agents
agent1 = AIAgentWithChatGPT('Agent1', api_key)
agent2 = AIAgentWithChatGPT('Agent2', api_key)

# Explains rules to both agents
explain_rules([agent1, agent2])

# Randomly assign roles
answerer, guesser = random.sample([agent1, agent2], 2)

# Starts the game
play_20_questions(answerer, guesser)


User, please enter the object to be guessed: Chair
Agent1 is the answerer.
Agent2 is the guesser.
Question 1: Is the object typically found indoors?
Answer: Yes
Press Enter to continue to the next question...ENTER
Question 2: Q: Is the object used for a specific purpose or function?
Answer: Yes
Press Enter to continue to the next question...enter
Question 3: Is the object commonly used in the kitchen?
Answer: No
Press Enter to continue to the next question...enter
Question 4: Does the object have electronic components?
Answer: No
Press Enter to continue to the next question...enter
Question 5: Is the object typically used for storage?
Answer: No
Press Enter to continue to the next question...enter
Question 6: Q: Is the object typically used in a living room or bedroom?
Answer: Yes
